In [8]:
import os
import torch
import sentencepiece
from transformers import T5Tokenizer, T5Model

from sklearn.metrics.pairwise import cosine_similarity

In [2]:
tokenizer = T5Tokenizer.from_pretrained("t5-base", cache_dir="/scratch/mbarlow6/.cache/")
model = T5Model.from_pretrained("t5-base", cache_dir="/scratch/mbarlow6/.cache/")

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

/home/mbarlow6/.local/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [5]:
def squeeze(word, tokenizer=tokenizer, model=model, debug=True):
    inputs = tokenizer(word, return_tensors="pt").input_ids
    decodes = tokenizer('', return_tensors="pt").input_ids  # want just </s>
    if debug:
        print('Tokens Requested:')
        print(tokenizer.batch_decode(inputs[0]))
    with torch.no_grad():
        outputs = model(input_ids=inputs, decoder_input_ids=decodes)
    last = outputs.last_hidden_state
    embeddings = torch.squeeze(last, dim=0)
    return embeddings


In [12]:
cosine_similarity(squeeze('The lawyer had a big presentation.'), squeeze('He argued with the judge.'))

Tokens Requested:
['The', 'lawyer', 'had', '', 'a', 'big', 'presentation', '.', '</s>']
Tokens Requested:
['He', '', 'argued', 'with', 'the', 'judge', '.', '</s>']


array([[0.9221202]], dtype=float32)